In [62]:
import psycopg2
from psycopg2 import sql

In [92]:
import os
from dotenv import load_dotenv

load_dotenv()

conn = psycopg2.connect(
    dbname=os.getenv('POSTGRES_DB'),  # Replace with your database name
    user=os.getenv('POSTGRES_USER'),       # Replace with your username
    password=os.getenv('POSTGRES_PASSWORD'),  # Replace with your password
    host=os.getenv('POSTGRES_HOST'),  # Replace with your host (e.g., localhost)
    port=os.getenv('PORT', '5432')      # Replace with your port if different
)


In [64]:
# Create a cursor object
cur = conn.cursor()

# Function to insert a new row
def insert_row(title, content, embeddings):
    insert_query = """
    INSERT INTO note_data (title, content, embeddings)
    VALUES (%s, %s, %s)
    """
    cur.execute(insert_query, (title, content, embeddings))
    conn.commit()

In [85]:
import markdown_parser
import os
import embedding_strings
import numpy as np

path = os.path.abspath(os.path.join(os.getcwd(), "../../../resources/sample-notes-markdown/TestData"))

for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        # Full path of each file
        # print(filename)
        # print(dirpath)
        file = dirpath + "\\" + filename
        # print(markdown_parser.markdown_plaintext(file))
        title = markdown_parser.markdown_plaintext(file)[0]
        content = markdown_parser.markdown_plaintext(file)[2]
        embeddings = embedding_strings.word_embeddings(markdown_parser.markdown_plaintext(file))
        avg_query_embedding = np.mean([emb for emb in embeddings.values()], axis=0)
        avg_query_embedding_list = avg_query_embedding.tolist()
        # insert_row(title, content, avg_query_embedding_list)
        print(len(avg_query_embedding_list))
        # print(embedding_strings.content_counter(markdown_parser.markdown_plaintext(file)))
        # print(embedding_strings.word_embeddings(markdown_parser.markdown_plaintext(file)))
        
        

384
384
384
384
384
384
384
384
384
384
384
384


In [81]:
embedding_strings.word_embeddings(["cc cc ccc ccc ccc c"])


{'cc': array([-1.41500905e-01,  7.55093843e-02, -1.79857403e-01,  1.38388023e-01,
         6.32529929e-02, -8.57721865e-02,  1.59166202e-01,  2.02324122e-01,
         8.91455933e-02, -5.44674210e-02,  3.97548676e-02, -3.37823540e-01,
        -4.62966189e-02,  8.40016007e-02, -6.70202672e-02,  8.20899196e-03,
        -4.54160348e-02,  7.87429139e-02, -1.33562550e-01, -2.42451970e-02,
        -1.43292710e-01,  3.87532599e-02, -2.86520664e-02,  5.48358336e-02,
        -1.05392173e-01,  1.41569689e-01, -1.24294728e-01,  6.60542697e-02,
        -1.20050469e-02, -2.25575432e-01, -3.70439738e-02,  6.98517114e-02,
         3.59550595e-01,  1.25891129e-02,  2.41729487e-02, -1.04381762e-01,
        -3.25781070e-02, -1.96169950e-02, -2.43261531e-02, -1.20328758e-02,
        -1.38733732e-02, -2.17967685e-02,  4.60535549e-02,  1.02042668e-02,
         7.81086385e-02,  1.42099410e-01,  1.03067219e-01, -4.15033959e-02,
        -3.38142514e-02,  8.83986577e-02, -2.02139225e-02,  2.88746450e-02,
      

In [86]:
import certifi
print(certifi.where())


d:\Anaconda\envs\stack\lib\site-packages\certifi\cacert.pem
